<a href="https://colab.research.google.com/github/mahsaghaderan99/DigikalaCommentAnalysis/blob/tokenizer/comment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation

In [4]:
!pip install stanfordnlp

     |████████████████████████████████| 163kB 20.6MB/s 


#Imports

In [5]:
import stanfordnlp
import numpy as np
import pandas as pd 

In [13]:
import gensim 
from gensim.models import Word2Vec 

#Data preparation

In [ ]:
#Save text file into drive

# !ls /content/drive/MyDrive
# !mkdir /content/drive/MyDrive/digikala
# !cp /content/digikala_dataset.txt /content/drive/MyDrive/digikala

In [ ]:
x = []
y = []
with open('/content/drive/MyDrive/digikala/digikala_dataset.txt','r') as f:
  for line in f:
    index = line.find('\t')
    dy = line[:index]
    dx = line[index+1:]
    x.append(dx)
    y.append(dy)


In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
punct = str.maketrans(dict.fromkeys(string.punctuation,' '))
nopunct_x = [s.translate(table) for s in x]
nopunct_x = [s.replace('\n',' ').replace('\t',' ') for s in x]

In [ ]:
data = {'label':y, 
        'comment':nopunct_x} 
df = pd.DataFrame(data) 
df 

,label,comment
0,0,لطفا رنگ سفید رو هم موجود کنید
1,0,با سلام؛ من همین الان میخواستم نوک مدادی بخرم ...
2,0,من یه روز گذاشتم شارژ 12 روز شارژ داشت
3,0,برای خاموش کردن نوار چراغ پایین گوشی از برنامه...
4,0,سلام بر دیجی کالا من میخواستم امروز بخرم ولی ا...
...,...,...
200756,0,لطفا موجود کنین
200757,-1,من 1 ساله دارمش واقعا بده. هنگ زیاد داغ شدن زی...
200758,0,اگه دو سیمشم بیاری دیجی کالا و نقد و بررسی هم ...
200759,1,سلام.من این گوشیو 2هفته پیش خریدم البته ایم بگ...


In [ ]:
#create csv file
# !touch /content/drive/MyDrive/digikala/digikala.csv

In [ ]:
df.to_csv ('/content/drive/MyDrive/digikala/digikala.csv', index = False, header=True)

#Tokenize

In [6]:
df = pd.read_csv('/content/drive/MyDrive/digikala/digikala.csv')

In [7]:
label = np.array(df['label'])
clean_com = np.array(df['comment'])

In [8]:
stanfordnlp.download('fa')

Using the default treebank "fa_seraji" for language "fa".
Would you like to download the models for: fa_seraji now? (Y/n)
y

Default download directory: /root/stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: /root/stanfordnlp_resources/fa_seraji_models.zip


100%|██████████| 226M/226M [01:14<00:00, 3.04MB/s]



Download complete.  Models saved to: /root/stanfordnlp_resources/fa_seraji_models.zip
Extracting models file for: fa_seraji
Cleaning up...Done.


In [9]:
nlp = stanfordnlp.Pipeline(processors='tokenize', lang='fa', tokenize_pretokenized=True)
tokens = []
for row in clean_com:
  doc = nlp(row)
  sentence = list(doc.sentences)[0]
  tokenz = [token.text for token in sentence.tokens]
  tokens.append(tokenz)

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/fa_seraji_models/fa_seraji_tokenizer.pt', 'pretokenized': True, 'lang': 'fa', 'shorthand': 'fa_seraji', 'mode': 'predict'}
Done loading processors!
---


In [41]:
model1 = gensim.models.Word2Vec(tokens, min_count = 1, size = 200, window = 7) 

In [42]:
model1.similarity('بد', 'عالی')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.36352932

In [43]:
model1.similarity('بو', 'عطر')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.27725998

In [44]:
model1.similarity('سفید', 'عالی')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.045049004

In [45]:
model1.similarity('بد', 'خش')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.19281656

In [46]:
model1.similarity('بد', 'تیره')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.16312863

In [47]:
model1.similarity('دارم', 'دوست')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


-0.15102327

In [48]:
model1.similarity('ندارم', 'دوست')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


-0.018434314

In [49]:
model1.similarity('عالی', 'مرسی')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


-0.09919945

In [50]:
model1.similarity('مرسی', 'ممنون')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.7369827

In [51]:
model1.similarity('تشکر', 'ممنون')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.55200577

In [52]:
model1.similarity('افتضاح', 'ممنون')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


-0.13374369

In [53]:
model1.similarity('مرسی', 'افتضاح')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


-0.12294457

In [54]:
model1.similarity('توقع', 'سفید')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


-0.15566441

In [55]:
model1.similarity('خوب', 'بد')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.57607144